# SQL Data Types, Aggregation, Sorting And Basic Calculations
## Group 1

- Adebowale Tosin
- Adetunji Emmanuel
- Adetunji Raheeemat
- Ajaye Othuke
- Akindele Yinka

## @Tosin

# Querying numerical data;

- Assessing the datatypes of table columns
- Numeric data
    - Integers 
    - Decimals

## Introduction

SQL databases consist of tables which in turn consist of rows and columns. Each column is allowed to store a single datatype and for the most pat static.

<img src="1.png" style=width:200px


This datatype is allocated to a given column based on:

1. The **type** of data that needs to be stored in the column.
2. The **size** (in bytes) required to store each datum in the column.


As such, the general rule of thumb is to use the smallest version of the data type that also has enough capacity to reliably support the data to be stored.

For this train, we will discuss some of the common datatypes in databases and SQL queries that will be useful when working with them.  

## Assessing the datatypes of table columns

In this section, we discuss how to find out the datatypes of columns in a given table. This will be useful when we want to use the WHERE clause (i.e. to verify the type of data in the column) or when we need to modify or add information to a given database table. 

First, let's load our sample database:

In [1]:
# load sql magics
%load_ext sql

# load chinook database
%sql sqlite:///chinook.db

Chinook db ER diagram:

![Chinook ERD](https://github.com/Explore-AI/Pictures/blob/master/sqlite-sample-database-color.jpg?raw=true)

_[Image source](https://www.sqlitetutorial.net/sqlite-sample-database/)_

We can show information (including column types) about a table in the database as follows:

**SQLite:**

In [2]:
%%sql

PRAGMA table_info(employees);

 * sqlite:///chinook.db
Done.


cid,name,type,notnull,dflt_value,pk
0,EmployeeId,INTEGER,1,None,1
1,LastName,NVARCHAR(20),1,None,0
2,FirstName,NVARCHAR(20),1,None,0
3,Title,NVARCHAR(30),0,None,0
4,ReportsTo,INTEGER,0,None,0
5,BirthDate,DATETIME,0,None,0
6,HireDate,DATETIME,0,None,0
7,Address,NVARCHAR(70),0,None,0
8,City,NVARCHAR(40),0,None,0
9,State,NVARCHAR(40),0,None,0


This query uses the [PRAGMA](https://www.sqlite.org/pragma.html) statement. This is useful for querying metadata of a database.

## 1. Numeric data
When using SQL in the context of data science, we will need to be familiar with numerical data in the form of integers and floats.

### 1.1. Integers
SQL offers multiple datatypes for storing integer values (negative and positive whole numbers). These integer datatypes will vary depending on the range, i.e., the interval of supported integers, and the nature of the table column. While it only supports 4 bytes of memory and integers between a given range (i.e. roughly $-2^{31}$ to $2^{31}$ ) it is sufficient for most datasets. SQL has other integer data types that vary depending on the storage size such as:

- `INTEGER` (or `INT` depending on the SQL engine used) - allocates 4 bytes of memory per integer and supports integers between the range $-2^{31}$ to $2^{31}-1$.
- `BIGINT`  - allocates 8 bytes of memory per integer and supports integers between the range $-2^{63}$ to $2^{63}-1$.
- `SMALLINT` - allocates 2 bytes of memory per integer and supports integers between the range $-2^{15}$ to $2^{15}-1$. For example, the employee age (this value will always be positive and seldom exceed 100).
- `SERIAL` - a special integer datatype that auto increments when rows are added to the table (useful for creating id columns). Like the `INTEGER` data type, the `SERIAL` data type is 4 bytes in size but only supports integers from $1$ to $2^{63}-1$. Furthermore, it also has `BIGSERIAL` and `SMALLSERIAL` variants which correspond to the `BIGINT` and `SMALLINT` data types respectively.

With this knowledge, let's look at the tracks table in the Chinook database:

In [3]:
%%sql

PRAGMA table_info(tracks);

 * sqlite:///chinook.db
Done.


cid,name,type,notnull,dflt_value,pk
0,TrackId,INTEGER,1,None,1
1,Name,NVARCHAR(200),1,None,0
2,AlbumId,INTEGER,0,None,0
3,MediaTypeId,INTEGER,1,None,0
4,GenreId,INTEGER,0,None,0
5,Composer,NVARCHAR(220),0,None,0
6,Milliseconds,INTEGER,1,None,0
7,Bytes,INTEGER,0,None,0
8,UnitPrice,"NUMERIC(10,2)",1,None,0


We can see that it contains 6 integer columns, the `TrackId`, `AlbumId`, `MediaTypeId`, `GenreId`, `Milliseconds`, and `Bytes` columns. 

Let's take a closer look at what these columns contain:

**Note, we limit our results to 10 rows for legibility.*

In [4]:
%%sql

SELECT TrackId, AlbumId, MediaTypeId, GenreId, Milliseconds, Bytes
FROM tracks
LIMIT 10;

 * sqlite:///chinook.db
Done.


TrackId,AlbumId,MediaTypeId,GenreId,Milliseconds,Bytes
1,1,1,1,343719,11170334
2,2,2,1,342562,5510424
3,3,2,1,230619,3990994
4,3,2,1,252051,4331779
5,3,2,1,375418,6290521
6,1,1,1,205662,6713451
7,1,1,1,233926,7636561
8,1,1,1,210834,6852860
9,1,1,1,203102,6599424
10,1,1,1,263497,8611245


As expected all these columns contain integers. However, most of them are column IDs, this means that we could generate them ourselves using the `SERIAL` integer types if we had to rebuild the database. A good way to analyse the properties of a given numerical column is to use summary statistics. 

Let's do this for the Bytes and the Milliseconds columns. 

In [5]:
%%sql

SELECT mt.Name, max(t.Bytes) AS "Bytes_Max", min(t.Bytes) AS "Bytes_Min", avg(t.Bytes) AS "Bytes_Mean", 
       max(t.Milliseconds) AS "Milliseconds_Max", min(t.Milliseconds) AS "Milliseconds_Min", avg(t.Milliseconds) "Milliseconds_Mean" 
FROM tracks AS t
LEFT JOIN media_types AS mt
ON mt.MediaTypeId = t.MediaTypeId
GROUP BY mt.Name;

 * sqlite:///chinook.db
Done.


Name,Bytes_Max,Bytes_Min,Bytes_Mean,Milliseconds_Max,Milliseconds_Min,Milliseconds_Mean
AAC audio file,6034098,2775071,4476793.818181818,366085,172710,276506.9090909091
MPEG audio file,52490554,38747,8630428.7656559,1612329,1071,265574.28872775217
Protected AAC audio file,11157785,1189062,4663795.573839663,672773,66639,281723.87341772154
Protected MPEG-4 video file,1059546140,20831818,420493713.0140187,5286953,112712,2342940.425233645
Purchased AAC audio file,16454937,2229617,8759372.42857143,493573,51780,260894.7142857143


Since the Bytes and Milliseconds are the same data type, the same limit applies to both of them. This means that we can't store tracks that are greater than $2^{31}-1$ bytes in size (i.e., 2 147 483 647 bytes or 2.15 gigabytes) and tracks that are longer than $2^{31}-1$ milliseconds (i.e., 597 hours). As you will see in later trains, it is important to know the datatypes of the columns we want to insert data into as well as the supported range of values.

### 1.2. Decimals

SQL also has datatypes for storing decimal numbers. These include:

- `DECIMAL` or `NUMERIC` - can store column values to user-specified precision (number of digits to the left and right of the decimal point) and variable storage size (based on specified precision). 
- `REAL` - allocates 4 bytes of memory per decimal and supports 7 decimal digits of fractional precision.
- `DOUBLE PRECISION` - allocates 8 bytes of memory per decimal and supports up to 15 decimal digits of fractional precision.

Let's take a look at the meta information of the invoices table:

In [5]:
%%sql

PRAGMA table_info(invoices);

 * sqlite:///chinook.db
Done.


cid,name,type,notnull,dflt_value,pk
0,InvoiceId,INTEGER,1,None,1
1,CustomerId,INTEGER,1,None,0
2,InvoiceDate,DATETIME,1,None,0
3,BillingAddress,NVARCHAR(70),0,None,0
4,BillingCity,NVARCHAR(40),0,None,0
5,BillingState,NVARCHAR(40),0,None,0
6,BillingCountry,NVARCHAR(40),0,None,0
7,BillingPostalCode,NVARCHAR(10),0,None,0
8,Total,"NUMERIC(10,2)",1,None,0


Evidently, the Total column is the only decimal column in the table. Its data type `NUMERIC(10,2)`, means that the column supports 10 digits to the left of the decimal point and 2 digits to the right of the decimal point.

Let's confirm this:

In [6]:
%%sql

SELECT Total
FROM invoices
LIMIT 10;

 * sqlite:///chinook.db
Done.


Total
1.98
3.96
5.94
8.91
13.86
0.99
1.98
1.98
3.96
5.94


The `NUMERIC` column type is a convenient choice for decimals since it allows the user to specify the desired precision, i.e., we can ask to store more or less numbers before or after the decimal point.

*Note: size is an extremely important factor when considering what data type to use for a given column. Particularly when dealing with numeric data. Using bigger datatypes not only affects the size of the database itself, but also the speed of calculations. This is especially true when dealing with high precision decimal values.*

## Conclusion

The concepts covered in this train will be more useful in cases where you have to create your own database tables. Although not comprehensively, we have covered: 

- How to view table meta information in SQLite - this is particularly useful in cases where we need to assess the datatypes of table columns.
- Common column data types in SQL such as: 
    - numerical data - `INTEGER`,`SERIAL`,`DECIMAL`, `REAL`, `DOUBLE PRECISION` 
    - text data - `CHAR(n)`,`VARCHAR(n)`, `VARCHAR`
    - date/time data - `DATE`, `DATETIME`, `INTERVAL`.
- How to construct queries around the above listed datatypes and useful built-in functions for each datatype.


## Source

- Working with different datatypes in SQL <br>© Explore Data Science Academy

- [SQL using MySQL : Numeric Data Types](https://www.youtube.com/watch?v=WDoWYriVCfs)